In [1]:
!pip install tweet-preprocessor
!pip install google-api-python-client

import pandas as pd
import preprocessor as p
import networkx as nx
import matplotlib.pyplot as plt
import tweepy
import json
import requests
import csv
import re
import sys
import urllib.parse

from sklearn.feature_extraction.text import CountVectorizer

from googleapiclient.discovery import build
from datetime import datetime

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=ff69049f2ccd1eaf2c9528b56967874fa1d8d11d0079b6264dbbdb9d0bee98ff
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [2]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Get News Data

In [0]:
timeline_data = {}
news_api_title = []
news_api_decription = []
news_api_date = []
news_api_url = []

def decode_title(dct):
    if "title" in dct:
        news_api_title.append(p.clean(dct["title"]))
    else:
        return [""]
def decode_description(dct):
    if "description" in dct:
        news_api_decription.append(p.clean(dct["description"]))
    else:
        return [""]
def decode_date(dct):
    if "publishedAt" in dct:
        news_api_date.append(dct["publishedAt"])
    else:
        return [""]
def decode_url(dct):
    if "url" in dct:
        news_api_url.append(dct["url"])
    else:
        return [""]

In [0]:
def getTimelineData():
    for i in range(len(news_api_title)):
      if news_api_date[i][:10] not in timeline_data:
        timeline_data[news_api_date[i][:10]] = []
        temp=[]
        temp.append(news_api_title[i])
        temp.append(news_api_url[i])
        timeline_data[news_api_date[i][:10]].append(temp)
      else:
        temp=[]
        temp.append(news_api_title[i])
        temp.append(news_api_url[i])
        timeline_data[news_api_date[i][:10]].append(temp)

In [0]:
def getGNewsAPI(query):

  keywords = urllib.parse.quote(query)
  url = "https://gnews.io/api/v3/search?q="+keywords+"&token=a6419076f909fc74c42016c6bebf0755"
  response = requests.get(url)
  gNewsApi_json = json.dumps(response.json(), indent = 4,sort_keys=True)
  print(gNewsApi_json)
  # Decode title
  parsed = json.loads(gNewsApi_json,object_hook=decode_title)
  # Decode description
  parsed = json.loads(gNewsApi_json,object_hook=decode_description)
  # Decode date
  parsed = json.loads(gNewsApi_json,object_hook=decode_date)
  # Decode url
  parsed = json.loads(gNewsApi_json,object_hook=decode_url)
  getTimelineData()

news_data = getGNewsAPI("coronavirus")

{
    "articleCount": 10,
    "articles": [
        {
            "description": "U.S. Senate Majority Leader Mitch McConnell (R-KY) arrives inside the U.S. Capitol after it was announced U.S. congressional leaders and the White House agreed on nearly $500 bill ...",
            "image": "http://www.bing.com/th?id=ON.D2B1FD1AD2A3A28459945E0BE2AFF915",
            "publishedAt": "2020-04-22 15:10:00 UTC",
            "source": {
                "name": "U.S. News & World Report",
                "url": "https://www.usnews.com"
            },
            "title": "Top U.S. Senate Republican Says Aid for States Needs to Focus on Coronavirus",
            "url": "https://www.usnews.com/news/us/articles/2020-04-22/top-us-senate-republican-says-aid-for-states-needs-to-focus-on-coronavirus"
        },
        {
            "description": "As the number of coronavirus cases continues to surge worldwide, health experts are learning more about the extent of the damage COVID-19 can do. The lungs 

In [0]:
print(timeline_data)
print(len(timeline_data))
print(len(news_api_title))
print(len(news_api_url))
# print(len(news_api_title))
# print(len(news_api_url))

{'2020-04-22': [['Top U.S. Senate Republican Says Aid for States Needs to Focus on Coronavirus', 'https://www.usnews.com'], ['Understanding the extent of damage COVID-19 does to the body', 'https://www.usnews.com/news/us/articles/2020-04-22/top-us-senate-republican-says-aid-for-states-needs-to-focus-on-coronavirus'], ["Report: Sean Payton Told Saints Team Won't Have Offseason Program Amid COVID-19", 'https://www.cbsnews.com'], ['NFLPA director DeMaurice Smith: Flexibility, innovation needed as NFL prepares for draft, offseason amid COVID-19 pandemic', 'https://www.cbsnews.com/video/coronavirus-understanding-damage-covid-19-does-to-body/'], ['England cricketer wonders whether the bug he caught in South Africa in December was coronavirus', 'https://bleacherreport.com'], ['Daily coronavirus count dips, but Indiana official points out technical glitch', 'https://bleacherreport.com/articles/2887965-report-sean-payton-told-saints-team-wont-have-offseason-program-amid-covid-19'], ["Tiger Wood

## Check News Data

In [0]:
print(timeline_data)

{'2020-04-22': [['Top U.S. Senate Republican Says Aid for States Needs to Focus on Coronavirus', 'https://www.usnews.com'], ['Understanding the extent of damage COVID-19 does to the body', 'https://www.usnews.com/news/us/articles/2020-04-22/top-us-senate-republican-says-aid-for-states-needs-to-focus-on-coronavirus'], ["Report: Sean Payton Told Saints Team Won't Have Offseason Program Amid COVID-19", 'https://www.cbsnews.com'], ['NFLPA director DeMaurice Smith: Flexibility, innovation needed as NFL prepares for draft, offseason amid COVID-19 pandemic', 'https://www.cbsnews.com/video/coronavirus-understanding-damage-covid-19-does-to-body/'], ['England cricketer wonders whether the bug he caught in South Africa in December was coronavirus', 'https://bleacherreport.com'], ['Daily coronavirus count dips, but Indiana official points out technical glitch', 'https://bleacherreport.com/articles/2887965-report-sean-payton-told-saints-team-wont-have-offseason-program-amid-covid-19'], ["Tiger Wood

In [0]:
print(news_api_title)

['Top U.S. Senate Republican Says Aid for States Needs to Focus on Coronavirus', 'Understanding the extent of damage COVID-19 does to the body', "Report: Sean Payton Told Saints Team Won't Have Offseason Program Amid COVID-19", 'NFLPA director DeMaurice Smith: Flexibility, innovation needed as NFL prepares for draft, offseason amid COVID-19 pandemic', 'England cricketer wonders whether the bug he caught in South Africa in December was coronavirus', 'Daily coronavirus count dips, but Indiana official points out technical glitch', "Tiger Woods, Phil Mickelson to golf with Peyton Manning, Tom Brady for 'The Match' to benefit coronavirus", "NY to launch coronavirus tracing with Bloomberg's help", 'California governor orders autopsies back to December to find out how long coronavirus has been in the state', 'Corrine Brown released from Central Florida prison over COVID-19 concerns: report']


In [0]:
print(news_api_date)

['2020-04-22 15:10:00 UTC', '2020-04-22 15:10:00 UTC', '2020-04-22 15:07:00 UTC', '2020-04-22 14:53:00 UTC', '2020-04-22 14:43:00 UTC', '2020-04-22 14:43:00 UTC', '2020-04-22 14:39:00 UTC', '2020-04-22 14:35:00 UTC', '2020-04-22 14:33:00 UTC', '2020-04-22 14:31:00 UTC']


## Split the URLs into News Source URLs and Article URLs

In [0]:
print(news_api_url)

['https://www.usnews.com', 'https://www.usnews.com/news/us/articles/2020-04-22/top-us-senate-republican-says-aid-for-states-needs-to-focus-on-coronavirus', 'https://www.cbsnews.com', 'https://www.cbsnews.com/video/coronavirus-understanding-damage-covid-19-does-to-body/', 'https://bleacherreport.com', 'https://bleacherreport.com/articles/2887965-report-sean-payton-told-saints-team-wont-have-offseason-program-amid-covid-19', 'https://www.azcentral.com', 'https://www.azcentral.com/story/sports/nfl/draft/2020/04/22/nfl-coronavirus-covid-19-pandemic-season-schedule-draft-offseason/3007743001/', 'https://www.irishexaminer.com', 'https://www.irishexaminer.com/breakingnews/sport/england-cricketer-wonders-whether-the-bug-he-caught-in-south-africa-in-december-was-coronavirus-995596.html', 'https://www.indystar.com', 'https://www.indystar.com/story/news/health/2020/04/22/indiana-coronavirus-technical-glitch-lowers-states-daily-count/3007498001/', 'https://www.cbssports.com', 'https://www.cbssport

In [0]:
article_urls = news_api_url[1::2]
print(article_urls)

['https://www.usnews.com/news/us/articles/2020-04-22/top-us-senate-republican-says-aid-for-states-needs-to-focus-on-coronavirus', 'https://www.cbsnews.com/video/coronavirus-understanding-damage-covid-19-does-to-body/', 'https://bleacherreport.com/articles/2887965-report-sean-payton-told-saints-team-wont-have-offseason-program-amid-covid-19', 'https://www.azcentral.com/story/sports/nfl/draft/2020/04/22/nfl-coronavirus-covid-19-pandemic-season-schedule-draft-offseason/3007743001/', 'https://www.irishexaminer.com/breakingnews/sport/england-cricketer-wonders-whether-the-bug-he-caught-in-south-africa-in-december-was-coronavirus-995596.html', 'https://www.indystar.com/story/news/health/2020/04/22/indiana-coronavirus-technical-glitch-lowers-states-daily-count/3007498001/', 'https://www.cbssports.com/golf/news/tiger-woods-phil-mickelson-to-golf-with-peyton-manning-tom-brady-for-the-match-to-benefit-coronavirus/', 'https://www.newsday.com/news/health/coronavirus/cuomo-bloomberg-virus-testing-1.

In [0]:
source_urls = news_api_url[0::2]
print(source_urls)

['https://www.usnews.com', 'https://www.cbsnews.com', 'https://bleacherreport.com', 'https://www.azcentral.com', 'https://www.irishexaminer.com', 'https://www.indystar.com', 'https://www.cbssports.com', 'https://www.newsday.com', 'https://thehill.com', 'https://www.orlandosentinel.com']


# Get News Article Text

In [0]:
import requests
from bs4 import BeautifulSoup
def getArticle(url):
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)

    article_text = ''
    article = soup.findAll('p')
    for element in article:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

In [0]:
article_corpus = getArticle("https://www.reuters.com/article/us-health-coronavirus-usa-trump-idUSKBN20Z04U")
print(article_corpus)


5 Min Read
WASHINGTON (Reuters) - President Donald Trump on Wednesday imposed sweeping restrictions to prevent people from 26 European countries from traveling to the United States for a month as he responded to mounting pressure to take action against the spread of the coronavirus. 
The president took the dramatic step in a somber Oval Office speech as he battled to address the health and economic shocks to Americans from the sometimes fatal virus and responded to criticism he has not taken the threat seriously enough. 
The travel order does not apply to the United Kingdom and Ireland, and does not apply to American citizens. 
Trump also announced several economic steps aimed at cushioning the blow to American businesses struggling with a sudden loss of consumer demand. 
“This is the most aggressive and comprehensive effort to confront a foreign virus in modern history,” he said. 
“I am confident that by counting and continuing to take these tough measures, we will significantly redu

# LDA Classifier

In [0]:
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [0]:
stemmer = SnowballStemmer("english")

def stemming_lemmatization(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def lemmatize_tokenization(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(stemming_lemmatization(token))
    return result

## Training Using a Predefined Set of News Articles

In [67]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_articles = fetch_20newsgroups(subset='train', shuffle = True)

In [0]:
print(list(newsgroups_articles.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [0]:
print(len(newsgroups_articles.data))

11314


In [68]:
# Reduced the size of the training data for faster processing. (Subject to change later.)
training_data = newsgroups_articles.data[:1000]
print(len(training_data))

1000


In [0]:
processed_newsgroups_docs = []

for doc in training_data:
    processed_newsgroups_docs.append(lemmatize_tokenization(doc))

In [0]:
print(processed_newsgroups_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [0]:
dictionary = gensim.corpora.Dictionary(processed_newsgroups_docs)

In [84]:
newsgroups_bow = [dictionary.doc2bow(doc) for doc in processed_newsgroups_docs]
print(newsgroups_bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(13, 1), (18, 1), (24, 1), (25, 1), (27, 1), (34, 1), (36, 1), (38, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 5), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 2), (59, 2), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 3), (87, 1), (88, 4)], [(9, 2), (14, 2), (15, 1), (18, 2), (19, 2), (25, 1), (27, 1), (34, 2), (36, 1), (38, 1), (39, 1), (43, 1), (54, 2), (58, 2), (62, 1

In [0]:
lda_model =  gensim.models.LdaMulticore(newsgroups_bow, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.010*"write" + 0.010*"subject" + 0.010*"line" + 0.009*"organ" + 0.007*"articl" + 0.006*"think" + 0.005*"mean" + 0.005*"univers" + 0.005*"post" + 0.005*"know"


Topic: 1 
Words: 0.009*"line" + 0.009*"subject" + 0.008*"organ" + 0.006*"write" + 0.005*"know" + 0.005*"post" + 0.005*"nntp" + 0.005*"host" + 0.005*"articl" + 0.004*"captain"


Topic: 2 
Words: 0.008*"line" + 0.008*"organ" + 0.008*"subject" + 0.007*"write" + 0.006*"father" + 0.005*"articl" + 0.005*"think" + 0.004*"spirit" + 0.004*"like" + 0.004*"believ"


Topic: 3 
Words: 0.009*"line" + 0.009*"subject" + 0.008*"organ" + 0.006*"write" + 0.005*"like" + 0.005*"need" + 0.004*"team" + 0.004*"articl" + 0.004*"think" + 0.004*"know"


Topic: 4 
Words: 0.010*"line" + 0.009*"organ" + 0.009*"subject" + 0.007*"write" + 0.006*"articl" + 0.005*"post" + 0.005*"univers" + 0.004*"year" + 0.004*"softwar" + 0.004*"peopl"


Topic: 5 
Words: 0.010*"argument" + 0.009*"fallaci" + 0.008*"subject" + 0.008*"line" + 0.007*"write" + 0.007

## Test using the Reuters Article

In [0]:
article_bow = dictionary.doc2bow(lemmatize_tokenization(article_corpus))

In [0]:
for index, score in sorted(lda_model[article_bow], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.2845754027366638	 Topic: 0.012*"armenian" + 0.009*"line" + 0.008*"subject" + 0.007*"organ" + 0.006*"peopl"
Score: 0.15173424780368805	 Topic: 0.010*"line" + 0.009*"organ" + 0.009*"subject" + 0.007*"write" + 0.006*"articl"
Score: 0.13484860956668854	 Topic: 0.009*"line" + 0.009*"subject" + 0.008*"organ" + 0.006*"write" + 0.005*"like"
Score: 0.10158166289329529	 Topic: 0.010*"subject" + 0.010*"line" + 0.009*"organ" + 0.008*"write" + 0.007*"post"
Score: 0.09635455906391144	 Topic: 0.008*"line" + 0.008*"organ" + 0.008*"subject" + 0.007*"write" + 0.006*"father"
Score: 0.0773455873131752	 Topic: 0.010*"write" + 0.010*"subject" + 0.010*"line" + 0.009*"organ" + 0.007*"articl"
Score: 0.058848559856414795	 Topic: 0.009*"scsi" + 0.009*"line" + 0.009*"subject" + 0.008*"organ" + 0.008*"post"
Score: 0.04788335785269737	 Topic: 0.009*"line" + 0.009*"subject" + 0.008*"organ" + 0.006*"write" + 0.005*"know"
Score: 0.027140403166413307	 Topic: 0.010*"argument" + 0.009*"fallaci" + 0.008*"subject"

In [0]:
for index, score in sorted(lda_model[article_bow], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.2845748960971832	 Topic: 8
Score: 0.15173441171646118	 Topic: 4
Score: 0.13485296070575714	 Topic: 3
Score: 0.1015847772359848	 Topic: 6
Score: 0.09635524451732635	 Topic: 2
Score: 0.07735032588243484	 Topic: 0
Score: 0.05884711071848869	 Topic: 7
Score: 0.04788580536842346	 Topic: 1
Score: 0.02712528593838215	 Topic: 5
Score: 0.01968914084136486	 Topic: 9


In [0]:
topic_result = sorted(lda_model[article_bow], key=lambda tup: -1*tup[1])
print(topic_result)

[(8, 0.2845746), (4, 0.15173614), (3, 0.13484372), (6, 0.10158341), (2, 0.096354626), (0, 0.07735561), (7, 0.058847748), (1, 0.047884367), (5, 0.027131477), (9, 0.01968835)]


# Create new Table to store KNN Data

In [0]:
news_articles_data = pd.DataFrame(columns=['Date','Source','Topic','Score'])
print(news_articles_data)

Empty DataFrame
Columns: [Date, Source, Topic, Score]
Index: []


## Perform LDA classification on Articles to get topic number

In [0]:
article_corpus_list = []
for url in article_urls:
  article_corpus_list.append(getArticle(url))
#print(article_corpus_list) 

In [0]:
processed_articles = []
for article in article_corpus_list:
    processed_articles.append(lemmatize_tokenization(article))
print(processed_articles)

[['refer'], ['watch', 'cbsn', 'live', 'copyright', 'interact', 'right', 'reserv'], ['orlean', 'saint', 'head', 'coach', 'sean', 'payton', 'report', 'tell', 'player', 'wednesday', 'team', 'hold', 'offseason', 'program', 'amid', 'coronavirus', 'pandem', 'espn', 'dianna', 'russini', 'report', 'payton', 'say', 'decis', 'stand', 'virtual', 'ota', 'chang', 'schedul', 'expect', 'team', 'wide', 'gather', 'kind', 'summer', 'earliest', 'saint', 'player', 'tell', 'virtual', 'workout', 'onlin', 'meet', 'workout', 'facil', 'allow', 'juli', 'train', 'camp', 'best', 'shape', 'life', 'tell', 'care', 'famili', 'health', 'readi', 'summer', 'saint', 'payton', 'test', 'posit', 'covid', 'march', 'recov', 'diseas', 'year', 'california', 'nativ', 'say', 'march', 'appear', 'radio', 'espn', 'mike', 'triplett', 'clear', 'yesterday', 'process', 'spend', 'time', 'tri', 'learn', 'certain', 'see', 'news', 'fortun', 'stay', 'insid', 'like', 'way', 'pass', 'time', 'competit', 'committe', 'meet', 'teleconfer', 'feel',

In [0]:
article_dictionary = gensim.corpora.Dictionary(processed_articles)

In [0]:
article_bow_corpus = [article_dictionary.doc2bow(doc) for doc in processed_articles]
print(article_bow_corpus)

[[(0, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(7, 2), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 2), (80, 1), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 3), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 3), (97, 1), (98, 1), (99, 3), (100, 1), (101, 1), (102, 1), (103, 1), (104, 3), (105, 1), (106, 1), (107, 1), (108, 1), (109, 

In [0]:
article_lda_model =  gensim.models.LdaMulticore(article_bow_corpus, 
                                   num_topics = 10, 
                                   id2word = article_dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [0]:
for idx, topic in article_lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.011*"right" + 0.011*"live" + 0.011*"interact" + 0.011*"reserv" + 0.011*"watch" + 0.011*"copyright" + 0.011*"cbsn" + 0.001*"player" + 0.001*"team" + 0.001*"say"


Topic: 1 
Words: 0.024*"play" + 0.020*"sport" + 0.014*"match" + 0.014*"mickelson" + 0.014*"event" + 0.014*"golf" + 0.010*"tiger" + 0.010*"tour" + 0.010*"cover" + 0.010*"wood"


Topic: 2 
Words: 0.001*"team" + 0.001*"player" + 0.001*"draft" + 0.001*"know" + 0.001*"smith" + 0.001*"say" + 0.001*"inform" + 0.001*"best" + 0.001*"think" + 0.001*"event"


Topic: 3 
Words: 0.001*"player" + 0.001*"team" + 0.001*"say" + 0.001*"test" + 0.001*"know" + 0.001*"smith" + 0.001*"inform" + 0.001*"facebook" + 0.001*"think" + 0.001*"health"


Topic: 4 
Words: 0.031*"releas" + 0.031*"prison" + 0.031*"brown" + 0.014*"report" + 0.014*"news" + 0.014*"year" + 0.014*"corrin" + 0.014*"fraud" + 0.009*"write" + 0.009*"counti"


Topic: 5 
Words: 0.025*"player" + 0.022*"team" + 0.017*"smith" + 0.016*"draft" + 0.014*"inform" + 0.013*"know"

In [0]:
article_bow_corpus = [dictionary.doc2bow(doc) for doc in processed_articles]
print(article_bow_corpus)

[[(437, 1)], [(246, 1), (616, 1), (2076, 1), (2497, 1), (3577, 1), (7093, 1)], [(5, 1), (14, 1), (15, 1), (19, 1), (40, 1), (76, 2), (90, 1), (95, 1), (96, 1), (110, 1), (111, 1), (122, 1), (127, 1), (128, 2), (133, 1), (134, 1), (141, 1), (161, 1), (163, 2), (168, 5), (195, 1), (221, 1), (230, 1), (243, 1), (252, 3), (265, 2), (274, 1), (338, 4), (378, 1), (382, 1), (383, 1), (388, 1), (436, 1), (442, 1), (454, 1), (458, 1), (470, 1), (595, 1), (611, 1), (613, 1), (681, 2), (703, 1), (761, 3), (765, 1), (805, 1), (823, 3), (835, 1), (865, 2), (877, 1), (880, 3), (898, 2), (906, 1), (954, 1), (981, 3), (1011, 2), (1016, 1), (1018, 1), (1020, 1), (1026, 1), (1078, 1), (1127, 1), (1129, 2), (1148, 1), (1153, 1), (1205, 1), (1294, 2), (1306, 1), (1314, 1), (1487, 1), (1512, 1), (1533, 1), (1534, 1), (1567, 1), (1690, 2), (1726, 2), (1746, 2), (1918, 1), (1920, 1), (1986, 1), (2010, 2), (2053, 1), (2107, 1), (2115, 1), (2195, 1), (2233, 1), (2262, 1), (2352, 1), (2385, 1), (2386, 1), (2417

In [0]:
article_lda_model_dict =  gensim.models.LdaMulticore(article_bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [0]:
for idx, topic in article_lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.000*"releas" + 0.000*"prison" + 0.000*"brown" + 0.000*"news" + 0.000*"time" + 0.000*"year" + 0.000*"tell" + 0.000*"program" + 0.000*"fraud" + 0.000*"wednesday"


Topic: 1 
Words: 0.006*"symptom" + 0.004*"ireland" + 0.003*"england" + 0.003*"time" + 0.003*"sport" + 0.003*"like" + 0.003*"africa" + 0.003*"examin" + 0.003*"south" + 0.002*"test"


Topic: 2 
Words: 0.011*"player" + 0.010*"team" + 0.007*"draft" + 0.006*"say" + 0.005*"smith" + 0.005*"know" + 0.005*"time" + 0.005*"inform" + 0.004*"think" + 0.004*"leagu"


Topic: 3 
Words: 0.000*"say" + 0.000*"time" + 0.000*"player" + 0.000*"symptom" + 0.000*"team" + 0.000*"draft" + 0.000*"test" + 0.000*"like" + 0.000*"know" + 0.000*"schedul"


Topic: 4 
Words: 0.000*"refer" + 0.000*"say" + 0.000*"player" + 0.000*"team" + 0.000*"draft" + 0.000*"time" + 0.000*"know" + 0.000*"sport" + 0.000*"smith" + 0.000*"symptom"


Topic: 5 
Words: 0.006*"play" + 0.005*"sport" + 0.003*"event" + 0.003*"match" + 0.003*"golf" + 0.003*"tour" + 0.0

In [0]:
article_topic_list = []
article_score_list = []
for article_vector in article_bow_corpus:
  for index, score in sorted(article_lda_model[article_vector], key=lambda tup: -1*tup[1]):
    article_topic_list.append(index)
    article_score_list.append(score)
    #print("Score: {}\t Topic: {}".format(score, index))
    break;

In [0]:
news_articles_data['Topic'] = article_topic_list
news_articles_data['Score'] = article_score_list

In [0]:
print(len(article_bow_corpus))
print(len(news_articles_data['Topic']))
print(len(news_articles_data['Score']))

10
10
10


##Test using Reuters Article

In [0]:
article_test = dictionary.doc2bow(lemmatize_tokenization(article_corpus))

In [0]:
for index, score in sorted(article_lda_model[article_test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.5222886204719543	 Topic: 9
Score: 0.2743585705757141	 Topic: 2
Score: 0.0817655399441719	 Topic: 1
Score: 0.07278605550527573	 Topic: 8
Score: 0.04732602462172508	 Topic: 5


## Convert Sources into Unique Numbers

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
source_urls_num = le.fit_transform(source_urls)
print(source_urls_num)

[9 3 0 2 6 5 4 7 1 8]


## Convert Date String to Number Format

In [0]:
import re
news_api_date_num = []
for date in news_api_date:
  check = re.sub('[^0-9]+', '', date)
  print(check)
  news_api_date_num.append(int(check))

20200422151000
20200422151000
20200422150700
20200422145300
20200422144300
20200422144300
20200422143900
20200422143500
20200422143300
20200422143100


In [0]:
print(news_api_date_num)

[20200422151000, 20200422151000, 20200422150700, 20200422145300, 20200422144300, 20200422144300, 20200422143900, 20200422143500, 20200422143300, 20200422143100]


## Assign data to the corresponding columns

In [0]:
news_articles_data['Date'] = news_api_date_num

In [0]:
news_articles_data['Source'] = source_urls_num

In [0]:
news_articles_data

,Date,Source,Topic,Score
0,20200422151000,9,9,0.550000
1,20200422151000,3,5,0.654615
2,20200422150700,0,2,0.899391
3,20200422145300,2,2,0.998097
4,20200422144300,6,1,0.994943
5,20200422144300,5,9,0.995454
6,20200422143900,4,5,0.994857
7,20200422143500,7,9,0.997594
8,20200422143300,1,9,0.993662
9,20200422143100,8,8,0.992105


#KNN

In [0]:
X = news_articles_data.iloc[:, :-2].values
y = news_articles_data.iloc[:, 2].values

In [0]:
print(X)

[[20200422151000              9]
 [20200422151000              3]
 [20200422150700              0]
 [20200422145300              2]
 [20200422144300              6]
 [20200422144300              5]
 [20200422143900              4]
 [20200422143500              7]
 [20200422143300              1]
 [20200422143100              8]]


In [0]:
print(y)

[9 5 2 2 1 9 5 9 9 8]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
print(y_pred)

[2 2 9 2]


In [0]:
print(y_test)

[5 9 5 2]


In [0]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.25


In [0]:
'''
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
'''

'\nfrom sklearn.metrics import classification_report, confusion_matrix\nprint(confusion_matrix(y_test, y_pred))\nprint(classification_report(y_test, y_pred))\n'

# Using only LDA to compare topics between 2 different news articles

In [0]:
single_news_url = "https://www.cnn.com/2020/04/25/health/us-coronavirus-saturday/index.html"

In [0]:
single_news_doc = []
single_news_doc.append(lemmatize_tokenization(getArticle(single_news_url)))

In [0]:
single_news_dictionary = gensim.corpora.Dictionary(single_news_doc)

In [66]:
print(single_news_dictionary)

Dictionary(54 unique tokens: ['advic', 'alexandra', 'amid', 'andon', 'april']...)


In [75]:
single_news_bow = [single_news_dictionary.doc2bow(doc) for doc in single_news_doc]
print(single_news_bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1)]]


In [0]:
single_news_lda_model =  gensim.models.LdaMulticore(single_news_bow, 
                                   num_topics = 5, 
                                   id2word = single_news_dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [83]:
for idx, topic in single_news_lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.019*"help" + 0.019*"coronavirus" + 0.019*"happen" + 0.019*"nicol" + 0.019*"tri" + 0.019*"meek" + 0.019*"danger" + 0.019*"natasha" + 0.019*"certain" + 0.019*"chen"


Topic: 1 
Words: 0.019*"coronavirus" + 0.019*"help" + 0.019*"get" + 0.019*"happen" + 0.019*"health" + 0.019*"isaac" + 0.019*"natasha" + 0.019*"unfold" + 0.019*"alexandra" + 0.019*"andon"


Topic: 2 
Words: 0.033*"help" + 0.033*"coronavirus" + 0.018*"run" + 0.018*"expert" + 0.018*"facebook" + 0.018*"maxouri" + 0.018*"interact" + 0.018*"dakin" + 0.018*"chat" + 0.018*"case"


Topic: 3 
Words: 0.019*"coronavirus" + 0.019*"help" + 0.019*"jason" + 0.019*"tri" + 0.019*"expert" + 0.019*"chavez" + 0.019*"get" + 0.019*"weekend" + 0.019*"messeng" + 0.019*"lindsay"


Topic: 4 
Words: 0.019*"coronavirus" + 0.019*"help" + 0.019*"hanna" + 0.019*"danger" + 0.019*"isaac" + 0.019*"natasha" + 0.019*"state" + 0.019*"risk" + 0.019*"alexandra" + 0.019*"crisi"




In [0]:
test_url_1 = "https://www.cbsnews.com/news/coronavirus-doctor-son-battling-virus-in-the-hospital/"
test_article_corpus_1 = getArticle(test_url_1)

In [0]:
test_article_bow_1 = single_news_dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_1))

In [80]:
for index, score in sorted(single_news_lda_model[test_article_bow_1], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, single_news_lda_model.print_topic(index, 5)))

Score: 0.9529345035552979	 Topic: 0.001*"post" + 0.001*"know" + 0.000*"line" + 0.000*"park" + 0.000*"look"
Score: 0.011766391806304455	 Topic: 0.000*"know" + 0.000*"post" + 0.000*"separ" + 0.000*"detail" + 0.000*"spec"
Score: 0.011766370385885239	 Topic: 0.000*"know" + 0.000*"post" + 0.000*"addit" + 0.000*"enlighten" + 0.000*"nntp"
Score: 0.011766351759433746	 Topic: 0.000*"post" + 0.000*"earli" + 0.000*"day" + 0.000*"rest" + 0.000*"addit"
Score: 0.011766350828111172	 Topic: 0.000*"post" + 0.000*"brief" + 0.000*"know" + 0.000*"small" + 0.000*"info"


In [81]:
for index, score in sorted(single_news_lda_model[test_article_bow_1], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.9529345035552979	 Topic: 3
Score: 0.01176639273762703	 Topic: 0
Score: 0.011766370385885239	 Topic: 1
Score: 0.011766352690756321	 Topic: 4
Score: 0.011766349896788597	 Topic: 2


In [0]:
test_url_2 = "https://www.cbsnews.com/news/coronavirus-spread-states-reopening/"
test_article_corpus_2 = getArticle(test_url_2)

In [0]:
test_article_bow_2 = single_news_dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_2))

In [55]:
for index, score in sorted(single_news_lda_model[test_article_bow_2], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, single_news_lda_model.print_topic(index, 5)))

Score: 0.9691982269287109	 Topic: 0.033*"coronavirus" + 0.033*"help" + 0.018*"advic" + 0.018*"spike" + 0.018*"chavez"


In [56]:
for index, score in sorted(single_news_lda_model[test_article_bow_2], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.9691982269287109	 Topic: 0


In [0]:
test_url_3 = "https://www.theguardian.com/us-news/2020/apr/20/us-protests-lockdown-coronavirus-cases-surge-warning"
test_article_corpus_3 = getArticle(test_url_3)

In [0]:
test_article_bow_3 = single_news_dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_3))

In [58]:
for index, score in sorted(single_news_lda_model[test_article_bow_3], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, single_news_lda_model.print_topic(index, 5)))

Score: 0.9733060002326965	 Topic: 0.033*"coronavirus" + 0.033*"help" + 0.018*"advic" + 0.018*"spike" + 0.018*"chavez"


In [59]:
for index, score in sorted(single_news_lda_model[test_article_bow_3], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.9733060598373413	 Topic: 0


In [0]:
test_url_4 = "https://www.cnbc.com/2020/04/24/sports-ticketing-will-look-different-as-teams-get-creative-without-fans.html"
test_article_corpus_4 = getArticle(test_url_4)

In [0]:
test_article_bow_4 = single_news_dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_4))

In [61]:
for index, score in sorted(single_news_lda_model[test_article_bow_4], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, single_news_lda_model.print_topic(index, 5)))

Score: 0.9783557057380676	 Topic: 0.033*"coronavirus" + 0.033*"help" + 0.018*"advic" + 0.018*"spike" + 0.018*"chavez"


In [62]:
for index, score in sorted(single_news_lda_model[test_article_bow_4], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.9783557057380676	 Topic: 0


In [0]:
# Only to test
print(dictionary)

In [0]:
new_lda_model = gensim.models.ldamodel.LdaModel(corpus=newsgroups_bow,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [98]:
# Test using new lda model
for idx, topic in new_lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.012*"play" + 0.008*"power" + 0.007*"smokeless" + 0.007*"buffalo" + 0.007*"player" + 0.006*"vancouv" + 0.006*"coli" + 0.006*"space" + 0.006*"nasa" + 0.006*"april"


Topic: 1 
Words: 0.020*"jesus" + 0.013*"christian" + 0.009*"think" + 0.009*"matthew" + 0.008*"know" + 0.008*"write" + 0.008*"peopl" + 0.007*"say" + 0.006*"moral" + 0.006*"argument"


Topic: 2 
Words: 0.010*"write" + 0.009*"know" + 0.009*"line" + 0.008*"like" + 0.008*"articl" + 0.008*"subject" + 0.007*"organ" + 0.007*"post" + 0.006*"time" + 0.006*"say"


Topic: 3 
Words: 0.010*"good" + 0.010*"year" + 0.009*"game" + 0.007*"articl" + 0.007*"write" + 0.006*"like" + 0.006*"line" + 0.006*"subject" + 0.006*"father" + 0.005*"organ"


Topic: 4 
Words: 0.014*"line" + 0.013*"organ" + 0.012*"subject" + 0.009*"write" + 0.009*"articl" + 0.008*"post" + 0.008*"year" + 0.006*"brian" + 0.006*"morri" + 0.005*"nntp"


Topic: 5 
Words: 0.008*"yeast" + 0.007*"philadelphia" + 0.006*"match" + 0.006*"contradict" + 0.005*"nasa" + 0

In [0]:
test_article_bow_2_x = dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_2))

In [101]:
for index, score in sorted(new_lda_model[test_article_bow_2_x], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.2727001905441284	 Topic: 2
Score: 0.2422049641609192	 Topic: 3
Score: 0.10445252060890198	 Topic: 6
Score: 0.09866520017385483	 Topic: 1
Score: 0.08435063064098358	 Topic: 4
Score: 0.08311180025339127	 Topic: 7
Score: 0.04970560595393181	 Topic: 0
Score: 0.045800499618053436	 Topic: 9
Score: 0.01082055363804102	 Topic: 8


In [0]:
test_article_bow_4_x = dictionary.doc2bow(lemmatize_tokenization(test_article_corpus_4))

In [104]:
for index, score in sorted(new_lda_model[test_article_bow_4_x], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, index))

Score: 0.3318111300468445	 Topic: 3
Score: 0.3023572862148285	 Topic: 2
Score: 0.09438008069992065	 Topic: 4
Score: 0.08453676849603653	 Topic: 7
Score: 0.04877018555998802	 Topic: 9
Score: 0.04356396943330765	 Topic: 1
Score: 0.030344508588314056	 Topic: 8
Score: 0.026741961017251015	 Topic: 0
Score: 0.02342669665813446	 Topic: 6
Score: 0.014067437499761581	 Topic: 5


# Future To-Do's



1.   Increase KNN Data Size - (Currently only 10 articles)
2.   

